In [1]:
from postprocessing import fabric_api

/Users/ra-mit/development/fabric/postprocessing/fabric_api.py:141: SyntaxWarning: name 'location_dic' is assigned to before global declaration
  global location_dic
/Users/ra-mit/development/fabric/postprocessing/fabric_api.py:143: SyntaxWarning: name 'inv_location_dic' is assigned to before global declaration
  global inv_location_dic
/Users/ra-mit/development/fabric/postprocessing/fabric_api.py:218: SyntaxWarning: name 'vectorizer' is assigned to before global declaration
  global vectorizer
Using TensorFlow backend.


In [2]:
from keras.models import load_model
mod = load_model("/Users/ra-mit/development/fabric/uns/ojal.h5")

In [34]:
from utils import prepare_sqa_data
import numpy as np
data = prepare_sqa_data.get_sqa()
# shuffle data
random_permutation = np.random.permutation(len(data))
data = np.asarray(data)
data = data[random_permutation]
# training, test
total_test = int(len(data) * 0.9)
train_stories = data[0:total_test]
test_stories = data[total_test::]

In [35]:
vocab = set()
for story, q, answer in train_stories:
    vocab |= set(story + q + answer)
for story, q, answer in test_stories:
    vocab |= set(story + q + answer)
vocab = sorted(vocab)
vocab_size = len(vocab) + 1
story_maxlen = 0
query_maxlen = 0
for story, q, _ in train_stories:
    if len(story) > story_maxlen:
        story_maxlen = len(story)
    if len(q) > query_maxlen:
        query_maxlen = len(q)
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))

In [36]:
from keras.preprocessing.sequence import pad_sequences
def vectorize_stories(data, word_idx, story_maxlen, query_maxlen):
    X = []
    Xq = []
    Y = []
    for story, query, answer in data:
        x = [word_idx[w] for w in story]
        xq = [word_idx[w] for w in query]
        # let's not forget that index 0 is reserved
        y = np.zeros(len(word_idx) + 1)
        for el in answer:
            y[word_idx[el]] = 1
        X.append(x)
        Xq.append(xq)
        Y.append(y)
    return (pad_sequences(X, maxlen=story_maxlen),
            pad_sequences(Xq, maxlen=query_maxlen), np.array(Y))

In [37]:
inputs_test, queries_test, answers_test = vectorize_stories(test_stories,
                                                            word_idx,
                                                            story_maxlen,
                                                            query_maxlen)

In [38]:
example = test_stories[0]
example
fact, q, a = example

In [39]:
a

['when', 'unreliable']

In [42]:
inputs_test[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  267,   83, 1377, 1329], dtype=int32)

In [43]:
queries_test[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0, 267,  83], dtype=int32)

In [44]:
answers_test[0]

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [61]:
f = np.asarray([inputs_test[0]])
q = np.asarray([queries_test[0]])
a = np.asarray([answers_test[0]])

In [62]:
list(f)

[array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  267,   83, 1377, 1329], dtype=int32)]

In [65]:
pa = mod.predict([f, q])

In [69]:
pa

array([[  7.57603048e-05,   3.18627601e-04,   7.38318704e-05, ...,
          1.44059581e-04,   8.10920435e-04,   8.49936987e-05]], dtype=float32)

In [70]:
def normalize_to_01_range(vector):
    min = np.min(vector)
    max = np.max(vector)
    normalized_vector = []
    for x in vector:
        z = (x - min) / (max - min)
        normalized_vector.append(z)
    return np.asarray(normalized_vector)

In [71]:
pan = normalize_to_01_range(pa)
pan

array([[ 0.0007622 ,  0.00663967,  0.00071553, ...,  0.00241506,
         0.01855334,  0.00098565]], dtype=float32)

In [78]:
max(pa)

0.041365955